# 1 モデルの準備

In [ ]:
!pip install transformers

In [2]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

In [4]:
print(bert_model.config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [5]:
print(bert_model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# 2 tokenizer

In [6]:
text_list = ["I'm reading a book",
             "I'd like to book a room",
             "I'd like to reserve a room",
             "Please reserve this table for us"]

In [7]:
encoding = tokenizer(text_list, max_length=10, padding="max_length", truncation=True, return_tensors="pt")
encoding

{'input_ids': tensor([[ 101, 1045, 1005, 1049, 3752, 1037, 2338,  102,    0,    0],
        [ 101, 1045, 1005, 1040, 2066, 2000, 2338, 1037, 2282,  102],
        [ 101, 1045, 1005, 1040, 2066, 2000, 3914, 1037, 2282,  102],
        [ 101, 3531, 3914, 2023, 2795, 2005, 2149,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

# 3 BERTと単語ベクトル

In [8]:
output = bert_model(**encoding, output_hidden_states=True)

In [9]:
len(output[2])
# output[2][層][文][単語]

13

In [10]:
text_list

["I'm reading a book",
 "I'd like to book a room",
 "I'd like to reserve a room",
 'Please reserve this table for us']

In [11]:
# 多義語
print(text_list[0], ",", text_list[1])
layer0_similarity = F.cosine_similarity(output[2][0][0][6].reshape(1, -1), output[2][0][1][6].reshape(1, -1))
layer12_similarity = F.cosine_similarity(output[2][-1][0][6].reshape(1, -1), output[2][-1][1][6].reshape(1, -1))
print(f"0層目: {round(layer0_similarity.item(), 3)}")
print(f"最終層目: {round(layer12_similarity.item(), 3)}")

I'm reading a book , I'd like to book a room
0層目: 1.0
最終層目: 0.359


In [12]:
# 類義語(同じ文脈)
print(text_list[1], ",", text_list[2])
layer0_similarity = F.cosine_similarity(output[2][0][1][6].reshape(1, -1), output[2][0][2][6].reshape(1, -1))
layer12_similarity = F.cosine_similarity(output[2][-1][1][6].reshape(1, -1), output[2][-1][2][6].reshape(1, -1))
print(f"0層目: {round(layer0_similarity.item(), 3)}")
print(f"最終層目: {round(layer12_similarity.item(), 3)}")

I'd like to book a room , I'd like to reserve a room
0層目: 0.096
最終層目: 0.717


In [13]:
# 類義語(違う文脈)
print(text_list[1], ",", text_list[3])
layer0_similarity = F.cosine_similarity(output[2][0][1][6].reshape(1, -1), output[2][0][3][2].reshape(1, -1))
layer12_similarity = F.cosine_similarity(output[2][-1][1][6].reshape(1, -1), output[2][-1][3][2].reshape(1, -1))
print(f"0層目: {round(layer0_similarity.item(), 3)}")
print(f"最終層目: {round(layer12_similarity.item(), 3)}")

I'd like to book a room , Please reserve this table for us
0層目: 0.026
最終層目: 0.415
